# Exercise 1.1 - Umbrella sampling

In [22]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import tqdm.contrib.concurrent

import soft_matter.sample
import soft_matter.energy
import soft_matter.dynamics

In [30]:
NUM_SEGMENTS = 16
A_SEGMENTS = 4
UMBRELLA_CONST = 10_000

np.random.seed(42)  # make experiments reproducible
data_dir = Path.cwd() / ".." / "data"
figure_dir = Path.cwd() / ".." / "report" / "figures"

sample_dist = .02
num_ensembles = int(3 / sample_dist) + 4
reaction_coordinates = np.linspace(0, 3, num=num_ensembles)
print(f"Planning on doing {num_ensembles} simulations.")

Planning on doing 15 simulations.


In [14]:
def energy_umbrella(polymer, constraint):
    polymer_energy = soft_matter.energy.total_bond_energy(polymer)
    constraint_shift = soft_matter.energy.polymer_pos_to_rad(polymer) - constraint
    umbrella_shift = UMBRELLA_CONST / 2 * np.sum(constraint_shift * constraint_shift)
    return polymer_energy + umbrella_shift

## Simulate the different ensembles

In [15]:
max_steps = 100_000
drop_start = 10_000
energy = energy_umbrella

def mc_sim(constraint):
    old_polymer = soft_matter.sample.sample_initial(NUM_SEGMENTS)
    ens = [old_polymer]
    energy_old = energy(old_polymer, constraint)
    for _ in range(max_steps * NUM_SEGMENTS):
        new_polymer = soft_matter.dynamics.polymer_step(old_polymer)
        energy_new = energy(new_polymer, constraint)

        acceptance_rate = np.exp(energy_old - energy_new)
        mc_number = np.random.rand()
        if mc_number < acceptance_rate:  # accepted
            ens.append(new_polymer)
            old_polymer = new_polymer
            energy_old = energy_new
        else:
            ens.append(old_polymer)  # rejected

    ens = np.array(ens[drop_start * NUM_SEGMENTS:])
    return ens

enss = tqdm.contrib.concurrent.process_map(mc_sim, reaction_coordinates)

  0%|          | 0/1600000 [00:00<?, ?it/s]

/tmp/ipykernel_1180353/2167198605.py:13: RuntimeWarning: overflow encountered in exp
  acceptance_rate = np.exp(energy_old - energy_new)
